In [1]:
import os
import scanpy as sc
import pandas as pd
import numpy as np

In [2]:
adata_dir = 'adata'
samples = [f[5:-5] for f in os.listdir(adata_dir)] 

In [3]:
genes = pd.read_csv('cnmf_genes.csv', header=None, index_col=0).index

In [4]:
adatas = []
for sample in samples:
    _adata = sc.read(os.path.join(adata_dir,'adata%s.h5ad'%sample))
    _adata.obs['patient'] = np.repeat(sample, _adata.n_obs)
    
    use_genes = _adata.var.index.intersection(genes)
    _adata = _adata[:,use_genes]
    
    adatas.append(_adata)

In [5]:
adata = adatas[0]
if len(adata)>1:
    adata = adata.concatenate(adatas[1::],batch_key='batch_number', join='outer', fill_value=0)

/Users/cpc45/opt/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/Users/cpc45/opt/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/Users/cpc45/opt/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0

In [6]:
# correct a few columns in var
vcol = adata.var.columns
cols_to_change = ['gene_ids','genome','mt','feature_types']
for newcol in cols_to_change:
    coi = [c for c in vcol if newcol in c]
    adata.var[newcol] = adata.var[coi].fillna(method='bfill', axis=1).iloc[:, 0]
    adata.var.drop(coi, axis=1, inplace=True)

bool_col = 'highly_variable'
coi = [c for c in vcol if bool_col in c]
adata.var[coi] = adata.var[coi].fillna(False)

In [7]:
adata.write('adata.h5ad')

In [12]:
adata

AnnData object with n_obs × n_vars = 68830 × 1500
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'patient', 'batch_number'
    var: 'n_cells_by_counts-0', 'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'n_cells-0', 'highly_variable-0', 'means-0', 'dispersions-0', 'dispersions_norm-0', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'log1p_total_counts-1', 'n_cells-1', 'highly_variable-1', 'means-1', 'dispersions-1', 'dispersions_norm-1', 'n_cells_by_counts-10', 'mean_counts-10', 'log1p_mean_counts-10', 'pct_dropout_by_counts-10', 'total_counts-10', 'log1p_total_counts-10', 'n_cells-10', '